In [1]:
import timeit
import pyspark
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.column import _to_java_column
from pyspark.sql.column import _to_seq

spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()

sc = spark.sparkContext

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# scored_videos - датасет на основе файла USvideos.csv с добавлением колонки, содержащей скор (показатель качества) видео

In [2]:
videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('../datasets/USvideos.csv')
videos.show(10)

+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|   video_id|               title|       channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|    Logan Paul Vlogs|         24|logan paul vlog|l...|4394029|320053|    5931|        46245|https://i.ytimg.c...|13.09|
|K4wEI5zhHB0|iPhone X — Introd...|               Apple|         28|Apple|iPhone 10|i...|7860119|185853|   26679|            0|https://i.ytimg.c...|13.09|
|cLdxuaxaQwc|         My Response|           PewDiePie|         22|              [none]|5845909|576597|   39774|       170708|https://i.ytimg.c...|13.09|
|WYYvHb03Eog|Apple iPhone X fi...|           The Verge|         28|apple iph

In [ ]:
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|   video_id|               title|       channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|    Logan Paul Vlogs|         24|logan paul vlog|l...|4394029|320053|    5931|        46245|https://i.ytimg.c...|13.09|
|K4wEI5zhHB0|iPhone X — Introd...|               Apple|         28|Apple|iPhone 10|i...|7860119|185853|   26679|            0|https://i.ytimg.c...|13.09|
|cLdxuaxaQwc|         My Response|           PewDiePie|         22|              [none]|5845909|576597|   39774|       170708|https://i.ytimg.c...|13.09|
|WYYvHb03Eog|Apple iPhone X fi...|           The Verge|         28|apple iphone x ha...|2642103| 24975|    4542|        12829|https://i.ytimg.c...|13.09|
|sjlHnJvXdQs|   iPhone X (parody)|          jacksfilms|         23|jacksfilms|parody...|1168130| 96666|     568|         6666|https://i.ytimg.c...|13.09|
|cMKX2tE5Luk|The Disaster Arti...|                 A24|          1|a24|a24 films|a24...|1311445| 34507|     544|         3040|https://i.ytimg.c...|13.09|
|8wNr-NQImFg|The Check In: HUD...|Late Night with S...|         23|Late night|Seth M...| 666169|  9985|     297|         1071|https://i.ytimg.c...|13.09|
|_HTXMhKWqnA|iPhone X Impressi...|    Marques Brownlee|         28|iPhone X|iphone x...|1728614| 74062|    2180|        15297|https://i.ytimg.c...|13.09|
|_ANP3HR1jsM|ATTACKED BY A POL...|    RomanAtwoodVlogs|         22|Roman Atwood|Roma...|1338533| 69687|     678|         5643|https://i.ytimg.c...|13.09|
|zgLtEob6X-Q|Honest Trailers -...|      Screen Junkies|          1|screenjunkies|scr...|1056891| 29943|     878|         4046|https://i.ytimg.c...|13.09|
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
only showing top 10 rows

In [3]:
videos.count()

7998

In [4]:
videos.dropDuplicates(('video_id',)).count()

2364

In [5]:
# отбираем последнюю запись для каждого видео
windowSpec  = Window.partitionBy("video_id").orderBy(col("date").desc())

videos_drop_dubl = videos.withColumn("row_number",row_number().over(windowSpec)) \
                         .where(col("row_number") == 1) \
                         .drop("row_number")

In [6]:
videos_drop_dubl.show(10)

+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|   video_id|               title|   channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|4yCkkOvIkUI|EXCLUSIVE: Zonniq...|        YBF Chic|         24|              [none]|   5662|    33|      21|           13|https://i.ytimg.c...|06.10|
|7TN09IP5JuI|Terry Crews Hallu...|  First We Feast|         26|First we feast|fw...|5066207|196868|    2083|        22920|https://i.ytimg.c...|10.10|
|Bo-qp-Zu0OY|Meeting Talking D...|    TouringPlans|         19|talking mickey|ta...|  21654|   179|       1|           30|https://i.ytimg.c...|30.09|
|JkqTeQHFoBY|Guardians of the ...|      Framestore|          1|marvel|vfx|CG|CGI...|  23300|   384| 

In [7]:
videos_drop_dubl.count()

2364

In [8]:
# Определяем веса для факторов
weight_views = 0.5
weight_comments = 0.3
weight_likes = 0.2
weight_dislikes = -0.2

In [9]:
# формула для расчета скора
score = videos['views'] * lit(weight_views) + videos['likes'] * lit(weight_likes) + videos['comment_total'] * lit(weight_comments) + videos['dislikes'] * lit(weight_dislikes) 

In [10]:
# scored_videos - датасет на основе файла USvideos.csv с добавлением колонки, содержащей скор 
scored_videos = videos_drop_dubl.withColumn('score', score)
scored_videos.show(10)

+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------------------+
|   video_id|               title|   channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|             score|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------------------+
|4yCkkOvIkUI|EXCLUSIVE: Zonniq...|        YBF Chic|         24|              [none]|   5662|    33|      21|           13|https://i.ytimg.c...|06.10|            2837.3|
|7TN09IP5JuI|Terry Crews Hallu...|  First We Feast|         26|First we feast|fw...|5066207|196868|    2083|        22920|https://i.ytimg.c...|10.10|         2578936.5|
|Bo-qp-Zu0OY|Meeting Talking D...|    TouringPlans|         19|talking mickey|ta...|  21654|   179|       1|           30|https://i.ytimg.c...|30.09|10871.

In [ ]:
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------------------+
|   video_id|               title|   channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|             score|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------------------+
|4yCkkOvIkUI|EXCLUSIVE: Zonniq...|        YBF Chic|         24|              [none]|   5662|    33|      21|           13|https://i.ytimg.c...|06.10|            2837.3|
|7TN09IP5JuI|Terry Crews Hallu...|  First We Feast|         26|First we feast|fw...|5066207|196868|    2083|        22920|https://i.ytimg.c...|10.10|         2578936.5|
|Bo-qp-Zu0OY|Meeting Talking D...|    TouringPlans|         19|talking mickey|ta...|  21654|   179|       1|           30|https://i.ytimg.c...|30.09|10871.599999999999|
|JkqTeQHFoBY|Guardians of the ...|      Framestore|          1|marvel|vfx|CG|CGI...|  23300|   384|       1|           31|https://i.ytimg.c...|30.09|11735.899999999998|
|K7pQsR8WFSo|Schlieren Imaging...|      Veritasium|         27|veritasium|scienc...| 633374| 29671|     284|         2200|https://i.ytimg.c...|05.10|          323224.4|
|RE-far-FvRs|PUPPIES FIRST BAT...|      VLOGTOWSKI|         22|vlog|family vlog|...| 323280| 13781|     292|         2024|https://i.ytimg.c...|12.10|164945.00000000003|
|TzyraAp3jaY|Martin Scorsese T...|     MasterClass|         27|Martin Scorsese|f...|  84257|   842|       1|           98|https://i.ytimg.c...|26.09|42326.100000000006|
|WQjO1mMCPg4|EVERY MCDONALD'S ...|           Smosh|         23|mcdonalds|drive t...|2318203| 64553|    6770|         7843|https://i.ytimg.c...|15.10|         1173011.0|
|_r5eTelhpmQ|Darius Rucker - L...|DariusRuckerVEVO|         10|Darius|Rucker|Lif...|  17783|  1072|      22|           68|https://i.ytimg.c...|30.09|            9121.9|
|aRgTLb5EbiQ| My Mama Wears Timbs|        Ari Fitz|         26|itsarifitz|ari fi...|  22165|  2643|      12|          259|https://i.ytimg.c...|16.10|11686.400000000001|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------------------+
only showing top 10 rows

# categories_score - датасет по категориям, в котором присутствуют следующие поля: Название категории (не id, он непонятный для аналитиков!). Медиана показателя score из датасета scored_videos по каждой категории.

In [11]:
# читаем json us_category_id
us_category_id = spark.read \
    .option("multiline","true") \
    .json('../datasets/US_category_id.json') \
    .select('items')
us_category_id.show(10, True)

+--------------------+
|               items|
+--------------------+
|[{"m2yskBQFythfE4...|
+--------------------+



In [12]:
# извлекаем id категорий
id = us_category_id.select(posexplode(col("items.id")).alias("pos", "id"))
id.show(5)

+---+---+
|pos| id|
+---+---+
|  0|  1|
|  1|  2|
|  2| 10|
|  3| 15|
|  4| 17|
+---+---+
only showing top 5 rows



In [13]:
# извлекаем категории
category = us_category_id.select(posexplode(col("items.snippet.title")).alias("pos", "title"))

In [14]:
# мапим категории и id, приводим id к типу Integer
category_id_name = id.join(category, "pos", "inner").select(col("id").cast(IntegerType()).alias("category_id"), col("title").alias("category_name"))
category_id_name.orderBy("id").show(10)

+-----------+----------------+
|category_id|   category_name|
+-----------+----------------+
|          1|Film & Animation|
|         10|           Music|
|         15|  Pets & Animals|
|         17|          Sports|
|         18|    Short Movies|
|         19| Travel & Events|
|          2|Autos & Vehicles|
|         20|          Gaming|
|         21|   Videoblogging|
|         22|  People & Blogs|
+-----------+----------------+
only showing top 10 rows



# JOIN без оптимизации

In [15]:
scored_videos_and_category = scored_videos.join(category_id_name, "category_id", "inner")
scored_videos_and_category.show(5)

+-----------+-----------+--------------------+-----------------+--------------------+------+-----+--------+-------------+--------------------+-----+------------------+--------------------+
|category_id|   video_id|               title|    channel_title|                tags| views|likes|dislikes|comment_total|      thumbnail_link| date|             score|       category_name|
+-----------+-----------+--------------------+-----------------+--------------------+------+-----+--------+-------------+--------------------+-----+------------------+--------------------+
|         28|CAQ2wWVlOuc|Jeff Bezos smashe...|     Tech Insider|Tech Insider|TI|T...| 66230|  661|     153|          203|https://i.ytimg.c...|22.10|           33277.5|Science & Technology|
|         28|9W0WPPpCFaM|You have more tha...|              Vox|vox.com|vox|expla...|209228| 7829|     237|          762|https://i.ytimg.c...|20.09|106361.00000000001|Science & Technology|
|         28|aawlU1IzuqI|iPhone 8 Charging...|     Appl

In [ ]:
+-----------+-----------+--------------------+-----------------+--------------------+------+-----+--------+-------------+--------------------+-----+------------------+--------------------+
|category_id|   video_id|               title|    channel_title|                tags| views|likes|dislikes|comment_total|      thumbnail_link| date|             score|       category_name|
+-----------+-----------+--------------------+-----------------+--------------------+------+-----+--------+-------------+--------------------+-----+------------------+--------------------+
|         28|CAQ2wWVlOuc|Jeff Bezos smashe...|     Tech Insider|Tech Insider|TI|T...| 66230|  661|     153|          203|https://i.ytimg.c...|22.10|           33277.5|Science & Technology|
|         28|9W0WPPpCFaM|You have more tha...|              Vox|vox.com|vox|expla...|209228| 7829|     237|          762|https://i.ytimg.c...|20.09|106361.00000000001|Science & Technology|
|         28|aawlU1IzuqI|iPhone 8 Charging...|     AppleInsider|iphone 8 5 watt|i...| 23366|  422|      17|           52|https://i.ytimg.c...|05.10|           11779.6|Science & Technology|
|         28|1L7JFN7tQLs|iPhone X + iPhone...|Jonathan Morrison|Apple|iPhone X|iP...|966001|25724|    1046|         4802|https://i.ytimg.c...|18.09|489376.69999999995|Science & Technology|
|         28|B9SptdjpJBQ| How To Learn Faster|      AsapSCIENCE|Learning tricks|w...|669851|34521|     346|         1372|https://i.ytimg.c...|30.09|          342172.1|Science & Technology|
+-----------+-----------+--------------------+-----------------+--------------------+------+-----+--------+-------------+--------------------+-----+------------------+--------------------+
only showing top 5 rows

In [16]:
scored_videos_and_category.count()

2364

# JOIN с оптимизацией Broadcast

In [ ]:
# датафрейм с маппингом категория - id является не большим, это своего рода справочник, который нужно приджойнить к основному 
# датафрейму с видео для того, чтобы получить понятные для аналитика названия категорий, поэтому для оптимизации джойна использован 
# бродкаст маленького датафрейма

In [17]:
scored_videos\
    .join(broadcast(category_id_name), "category_id", "inner")\
    .count()

2364

In [18]:
categories_score_pandas = scored_videos_and_category.pandas_api()

In [19]:
categories_score_pandas.head(10)

,category_id,video_id,title,channel_title,tags,views,likes,dislikes,comment_total,thumbnail_link,date,score,category_name
0,28,CAQ2wWVlOuc,Jeff Bezos smashes bottle of champagne while p...,Tech Insider,Tech Insider|TI|Tech|Science|Innovation|Digita...,66230,661,153,203,https://i.ytimg.com/vi/CAQ2wWVlOuc/default.jpg,22.10,33277.5,Science & Technology
1,28,9W0WPPpCFaM,You have more than five senses,Vox,vox.com|vox|explain|sense|perception|sophie sc...,209228,7829,237,762,https://i.ytimg.com/vi/9W0WPPpCFaM/default.jpg,20.09,106361.0,Science & Technology
2,28,aawlU1IzuqI,iPhone 8 Charging Speed Comparison - Is Fast-C...,AppleInsider,iphone 8 5 watt|iphone 8 fast-charging|iphone ...,23366,422,17,52,https://i.ytimg.com/vi/aawlU1IzuqI/default.jpg,05.10,11779.6,Science & Technology
3,28,1L7JFN7tQLs,iPhone X + iPhone 8 Hands on!,Jonathan Morrison,Apple|iPhone X|iPhone 8|iPhone X unboxing|Appl...,966001,25724,1046,4802,https://i.ytimg.com/vi/1L7JFN7tQLs/default.jpg,18.09,489376.7,Science & Technology
4,28,B9SptdjpJBQ,How To Learn Faster,AsapSCIENCE,Learning tricks|ways to learn faster|best ways...,669851,34521,346,1372,https://i.ytimg.com/vi/B9SptdjpJBQ/default.jpg,30.09,342172.1,Science & Technology
5,28,WYYvHb03Eog,Apple iPhone X first look,The Verge,iPhone X|iphone x hands on|iPhone X first look...,4804702,35719,7384,13127,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,19.09,2411956.1,Science & Technology
6,28,k9bYO7uSnrg,iPhone X hands on,TechCrunch,iphone x|apple iphone|face id|apple|hardware|h...,14577,160,19,51,https://i.ytimg.com/vi/k9bYO7uSnrg/default.jpg,13.09,7332.0,Science & Technology
7,28,F8i8pnZmk50,iPhone 8 vs 8 Plus Unboxing & Comparison!,Jonathan Morrison,iPhone 8|iPhone 8 Plus|iPhone 8 Unboxing|iPhon...,1363776,51439,1666,6150,https://i.ytimg.com/vi/F8i8pnZmk50/default.jpg,24.09,693687.6,Science & Technology
8,28,nOllkbo99SU,Best Fast Chargers for iPhone 8 / iPhone X - E...,iDB,Apple|iDownloadBlog|Andrew ohara|iDB|iphone|ip...,5660,220,5,44,https://i.ytimg.com/vi/nOllkbo99SU/default.jpg,05.10,2886.2,Science & Technology
9,28,d8U5funnyA8,Why Does the Mac Mini Exist in 2017?,Austin Evans,mac|mac mini|apple|mac os|cheap mac|budget mac...,917466,20937,1406,3085,https://i.ytimg.com/vi/d8U5funnyA8/default.jpg,22.09,463564.7,Science & Technology


In [20]:
categories_score = categories_score_pandas.groupby('category_name')['score'].median()

In [21]:
categories_score

category_name
Shows                      4395.1
Education                127538.4
Gaming                   126462.4
Entertainment            192165.7
Travel & Events          105931.7
Science & Technology     158050.0
Sports                    66815.1
Howto & Style            124715.6
Nonprofits & Activism     28155.5
Film & Animation         160161.3
People & Blogs           121552.8
News & Politics           83608.7
Pets & Animals            96185.0
Autos & Vehicles          84353.2
Music                    122129.2
Comedy                   398643.1
Name: score, dtype: float64

# popular_tags - датасет по самым популярным тэгам (название тэга + количество видео с этим тэгом)

In [22]:
def udfSplitTagsScalaWrapper(field):
    _splitTagsUDF = sc._jvm.CustomUDFs.splitTagsUDF()
    return Column(_splitTagsUDF.apply(_to_seq(sc, [field], _to_java_column)))

In [23]:
videos_drop_dubl_spl_tags1 = videos_drop_dubl.withColumn('Tag', explode(udfSplitTagsScalaWrapper(col('tags'))))
popular_tags1 = videos_drop_dubl_spl_tags1.groupBy('Tag').count()
popular_tags1.orderBy(col('count').desc()).show(10)

+------+-----+
|   Tag|count|
+------+-----+
| funny|  217|
|comedy|  163|
|[none]|  144|
|  2017|   93|
| humor|   92|
|how to|   84|
|makeup|   77|
| music|   74|
|  vlog|   73|
| video|   71|
+------+-----+
only showing top 10 rows



In [24]:
code_to_measure = """
videos_drop_dubl_spl_tags1 = videos_drop_dubl.withColumn('Tag', explode(udfSplitTagsScalaWrapper(col('tags'))))
popular_tags1 = videos_drop_dubl_spl_tags1.groupBy('Tag').count()
"""
setup_code = """
import pyspark
from pyspark.sql.functions import row_number, col, explode, udf
from pyspark.sql.column import _to_java_column
from pyspark.sql.column import _to_seq
from pyspark.sql import SparkSession, Column
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StringType


spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()
sc = spark.sparkContext

videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('../datasets/USvideos.csv')
windowSpec  = Window.partitionBy("video_id").orderBy(col("date").desc())
videos_drop_dubl = videos.withColumn("row_number",row_number().over(windowSpec)) \
                         .where(col("row_number") == 1) \
                         .drop("row_number")

def udfSplitTagsScalaWrapper(field):
    _splitTagsUDF = sc._jvm.CustomUDFs.splitTagsUDF()
    return Column(_splitTagsUDF.apply(_to_seq(sc, [field], _to_java_column)))
"""
execution_time = timeit.timeit(stmt=code_to_measure, setup=setup_code, number=1)
print(f"Execution time: {execution_time} seconds")

Execution time: 0.011523291992489249 seconds


In [25]:
# определяем UDF для разбивки тегов
def split_tags(tags):
    if tags:
        return tags.split('|')
    else:
        return []
# Create a UDF from the split_tags function
split_tags_udf = udf(split_tags, ArrayType(StringType()))

In [26]:
videos_drop_dubl_spl_tags = videos_drop_dubl.withColumn('Tag', explode(split_tags_udf(col('tags'))))
popular_tags = videos_drop_dubl_spl_tags.groupBy('Tag').count()
popular_tags.orderBy(col('count').desc()).show(10)

+------+-----+
|   Tag|count|
+------+-----+
| funny|  217|
|comedy|  163|
|[none]|  144|
|  2017|   93|
| humor|   92|
|how to|   84|
|makeup|   77|
| music|   74|
|  vlog|   73|
| video|   71|
+------+-----+
only showing top 10 rows



In [27]:
code_to_measure = """
videos_drop_dubl_spl_tags = videos_drop_dubl.withColumn('Tag', explode(split_tags_udf(col('tags'))))
popular_tags = videos_drop_dubl_spl_tags.groupBy('Tag').count()
"""
setup_code = """
import pyspark
from pyspark.sql.functions import row_number, col, explode, udf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StringType

spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()
sc = spark.sparkContext

videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('../datasets/USvideos.csv')
windowSpec  = Window.partitionBy("video_id").orderBy(col("date").desc())
videos_drop_dubl = videos.withColumn("row_number",row_number().over(windowSpec)) \
                         .where(col("row_number") == 1) \
                         .drop("row_number")


def split_tags(tags):
    if tags:
        return tags.split('|')
    else:
        return []
# Create a UDF from the split_tags function
split_tags_udf = udf(split_tags, ArrayType(StringType()))
"""
execution_time = timeit.timeit(stmt=code_to_measure, setup=setup_code, number=1)
print(f"Execution time: {execution_time} seconds")

Execution time: 0.012086332993931137 seconds


# Cамые интересные комментарии (топ-5) к видео про котов. “Видео про котов” - видео, у которого есть тэг “cat”.

In [28]:
# отсортировать видео у которых есть тег cat
videos_drop_dubl_spl_tags.show(5)

+-----------+--------------------+--------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+--------------+
|   video_id|               title| channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|           Tag|
+-----------+--------------------+--------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+--------------+
|4yCkkOvIkUI|EXCLUSIVE: Zonniq...|      YBF Chic|         24|              [none]|   5662|    33|      21|           13|https://i.ytimg.c...|06.10|        [none]|
|7TN09IP5JuI|Terry Crews Hallu...|First We Feast|         26|First we feast|fw...|5066207|196868|    2083|        22920|https://i.ytimg.c...|10.10|First we feast|
|7TN09IP5JuI|Terry Crews Hallu...|First We Feast|         26|First we feast|fw...|5066207|196868|    2083|        22920|https://i.ytimg.c...|10.10|           fwf|
|7TN09IP5JuI|Terry Cre

In [29]:
videos_drop_dubl_spl_tags.count()

43165

In [30]:
videos_cat_tag = videos_drop_dubl_spl_tags.where(lower((col("Tag"))) == "cat")

In [31]:
videos_cat_tag.count()

17

In [32]:
videos_cat_tag.show()

+-----------+------------------------------------+--------------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+---+
|   video_id|                               title|       channel_title|category_id|                tags|  views|likes|dislikes|comment_total|      thumbnail_link| date|Tag|
+-----------+------------------------------------+--------------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+---+
|7V1J_MDi9Lg|                  Husky's First Howl|       TheAmazingTen|         22|Husky's First How...| 892375|31960|     380|         1944|https://i.ytimg.c...|28.09|cat|
|tp9aQXDFHbY|                Cats Can Be A Rea...|         Simon's Cat|         15|cartoon|simons ca...| 103451| 6125|     107|          313|https://i.ytimg.c...|30.09|cat|
|tp9aQXDFHbY|                Cats Can Be A Rea...|         Simon's Cat|         15|cartoon|simons ca...| 103451| 6125|     107|        

In [ ]:
+-----------+------------------------------------+--------------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+---+
|   video_id|                               title|       channel_title|category_id|                tags|  views|likes|dislikes|comment_total|      thumbnail_link| date|Tag|
+-----------+------------------------------------+--------------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+---+
|7V1J_MDi9Lg|                  Husky's First Howl|       TheAmazingTen|         22|Husky's First How...| 892375|31960|     380|         1944|https://i.ytimg.c...|28.09|cat|
|tp9aQXDFHbY|                Cats Can Be A Rea...|         Simon's Cat|         15|cartoon|simons ca...| 103451| 6125|     107|          313|https://i.ytimg.c...|30.09|cat|
|tp9aQXDFHbY|                Cats Can Be A Rea...|         Simon's Cat|         15|cartoon|simons ca...| 103451| 6125|     107|          313|https://i.ytimg.c...|30.09|Cat|
|Vjc459T6wX8|更なるフィット感を追求するねこ。-...|            mugumogu|         15|Maru|cat|kitty|pe...|  95741| 4749|      20|          487|https://i.ytimg.c...|16.09|cat|
|-1fzGnFwz9M|                9 Things You Need...|         Simon's Cat|         15|cartoon|simons ca...| 189414| 7070|     112|          288|https://i.ytimg.c...|13.09|cat|
|-1fzGnFwz9M|                9 Things You Need...|         Simon's Cat|         15|cartoon|simons ca...| 189414| 7070|     112|          288|https://i.ytimg.c...|13.09|Cat|
|H7H0ZWwWKCw|                The Fire Whispere...|ExplosmEntertainment|         23|shorts|c&hshorts|...|1196355|48087|    1640|         3374|https://i.ytimg.c...|17.10|cat|
|xbBMVa2A68s|                Cat vs Dog - Best...|             TierZoo|         20|cat|dog|cute|gami...| 373506|21420|     696|         5412|https://i.ytimg.c...|21.10|cat|
|c5RsAXsZvI8|                7. Teach the Firs...|Clicker Learning ...|         15|cats|cat|kittens|...|   4962|    8|       0|            1|https://i.ytimg.c...|08.10|cat|
|BY3SLVNBkeo|                Things you didn't...|         Simon's Cat|         15|cartoon|simons ca...| 111901| 5230|      71|          349|https://i.ytimg.c...|27.09|cat|
|BY3SLVNBkeo|                Things you didn't...|         Simon's Cat|         15|cartoon|simons ca...| 111901| 5230|      71|          349|https://i.ytimg.c...|27.09|Cat|
|0Yhaei1S5oQ|                Japan's Ominous D...|             SciShow|         27|SciShow|science|H...| 307749|11202|     150|          800|https://i.ytimg.c...|14.09|cat|
|XA_jAJ-dWyQ|                 MEET MY NEW KITTEN!|       PsychoSoprano|         23|colleen ballinger...| 718813|49746|     399|         7263|https://i.ytimg.c...|07.10|cat|
|S9VIKOuZcds|                My Sweet Jax (Tri...|             Hot Dad|         23|cat|feline|pets|b...|  25037| 2846|      11|          537|https://i.ytimg.c...|22.10|cat|
|ruTB5jBGXsE|                Spider Cat - Simo...|         Simon's Cat|         15|cartoon|simons ca...| 576907|19225|     301|          831|https://i.ytimg.c...|12.10|cat|
|ruTB5jBGXsE|                Spider Cat - Simo...|         Simon's Cat|         15|cartoon|simons ca...| 576907|19225|     301|          831|https://i.ytimg.c...|12.10|Cat|
|ySon7Ou0vIM|                Kittens Come Out ...|         Jukin Media|         15|adorable|animal|b...|  36106|    0|       0|            0|https://i.ytimg.c...|10.10|cat|
+-----------+------------------------------------+--------------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+---+

In [33]:
# читаем датафрейм с комментариями
comments_schema = StructType([ \
    StructField("video_id", StringType(), True), \
    StructField("comment_text", StringType(), True), \
    StructField("likes", IntegerType(), True), \
    StructField("replies", IntegerType(), True)])
comments = spark.read.option('header', 'true').option("mode", "DROPMALFORMED").schema(comments_schema).csv('../datasets/UScomments.csv')
comments.show()

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|XpVt6Z1Gjjo|Logan Paul it's y...|    4|      0|
|XpVt6Z1Gjjo|I've been followi...|    3|      0|
|XpVt6Z1Gjjo|Say hi to Kong an...|    3|      0|
|XpVt6Z1Gjjo| MY FAN . attendance|    3|      0|
|XpVt6Z1Gjjo|         trending 😉|    3|      0|
|XpVt6Z1Gjjo|#1 on trending AY...|    3|      0|
|XpVt6Z1Gjjo|The end though 😭...|    4|      0|
|XpVt6Z1Gjjo|#1 trending!!!!!!!!!|    3|      0|
|XpVt6Z1Gjjo|Happy one year vl...|    3|      0|
|XpVt6Z1Gjjo|You and your shit...|    0|      0|
|XpVt6Z1Gjjo|There should be a...|    0|      0|
|XpVt6Z1Gjjo|Dear Logan, I rea...|    0|      0|
|XpVt6Z1Gjjo|Honestly Evan is ...|    0|      0|
|XpVt6Z1Gjjo|Casey is still be...|    0|      0|
|XpVt6Z1Gjjo|aw geez rick this...|    0|      0|
|XpVt6Z1Gjjo|He happy cause he...|    0|      0|
|XpVt6Z1Gjjo|Ayyyyoooo Logang ...|    1|      0|
|XpVt6Z1Gjjo|Bro y did

In [34]:
# отбираем комментарии у к видео про котов
video_cat_tag_comments = comments.join(videos_cat_tag, "video_id", "inner")

In [35]:
video_cat_tag_comments.count()

5901

In [36]:
# отображаем топ 5 комментариев по количеству лайков к видео про котов
video_cat_tag_comments.orderBy(comments["likes"].desc()).select("comment_text").show(5, False)

+----------------------------------------------------------------------------------+
|comment_text                                                                      |
+----------------------------------------------------------------------------------+
|The second I read this title in my notification, I started to giggle.             |
|talk about the ocean sunfish build                                                |
|talk about the ocean sunfish build                                                |
|talk about the ocean sunfish build                                                |
|I make interesting cartoons and I need your help! Go to the channel, rate my work!|
+----------------------------------------------------------------------------------+
only showing top 5 rows



# JOIN с оптимизацией Bucketing

In [ ]:
# использована оптимизация Bucketing, так как датафреймы USvideos.csv и UScomments.csv считаем огромными, соответственно
# даже при использовании фильтров по тегу Cat мы получим большое количество записей
# для ускорения join оба датафрейма разбиты на 10 бакетов по ключу соединения

In [38]:
comments.write \
    .bucketBy(10, 'video_id') \
    .saveAsTable('comments_bucketed', format='csv', mode='overwrite')
    
videos_cat_tag.write \
    .bucketBy(10, 'video_id') \
    .saveAsTable('videos_cat_tag_bucketed', format='csv', mode='overwrite')

In [39]:
comments_bucketed = spark.table('comments_bucketed')
videos_cat_tag_bucketed = spark.table('videos_cat_tag_bucketed')

comments_bucketed \
    .join(videos_cat_tag_bucketed, "video_id", "inner") \
    .count()

5901

In [ ]:
spark.stop()